In [ ]:
print('run many events')

In [ ]:
###here the mesh I used 7500 m which is very dense mesh with over 900000 elements and caused large burdern for the smoothing,
### thus decrease to 15 km is ok for P wave in the mantle with 2 elements to sample the wavelength or equal to 1 element for the crust 
### just for tests and illustration for my thesis

In [ ]:
####run the jupyter notebook for the tomography model 
import numpy as np
import matplotlib.pyplot as plt
import os
PROJECT='/data2/yjgao/data/DATASET/package_forthesis_small/'

In [ ]:
def launch_forward(dir_run,dir_event,simtype,modelfile,dir_storesyn):
    os.chdir(dir_run)
    os.chdir(dir_event)
    CHECK_FOLDER = os.path.isdir(dir_storesyn)
    if not CHECK_FOLDER:
        os.makedirs(dir_storesyn)
    from shutil import copyfile
    from distutils.dir_util import copy_tree
    copyfile(modelfile,'DATA/profile.xyz')
    copyfile('../run_this_example.sh','run_this_example.sh')
    os.system('ln -s specfem2d-devel/utils/change_simulation_type.pl')
    os.system('perl change_simulation_type.pl -f') 
    print('start forward modelling',dir_event)
    os.system('bash run_this_example.sh')
    print('finished forward modelling')
    ###store the synthetics to the dir observed or syn for backup
    copy_tree('OUTPUT_FILES',dir_storesyn)
    os.chdir(dir_run)
    #print(pwd)

In [ ]:
####compare smoothed model with 1D model 
modelfile=PROJECT+'MODELS/profile_obs.xyz'
from datetime import datetime
start=datetime.now()
#launch_forward(PROJECT,'EVENT1','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT3','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT4','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT5','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT6','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT7','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT8','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT9','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT10','forward',modelfile,'REF_SEM')
#launch_forward(PROJECT,'EVENT11','forward',modelfile,'REF_SEM')
launch_forward(PROJECT,'EVENT12','forward',modelfile,'REF_SEM')
launch_forward(PROJECT,'EVENT13','forward',modelfile,'REF_SEM')
print (datetime.now()-start)


In [ ]:
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
from datetime import datetime
start=datetime.now()
#launch_forward(PROJECT,'EVENT1','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT2','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT3','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT4','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT5','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT6','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT7','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT8','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT9','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT10','forward',modelfile,'REF_SEM_smooth')
#launch_forward(PROJECT,'EVENT11','forward',modelfile,'REF_SEM_smooth')
launch_forward(PROJECT,'EVENT12','forward',modelfile,'REF_SEM_smooth')
launch_forward(PROJECT,'EVENT13','forward',modelfile,'REF_SEM_smooth')
print (datetime.now()-start)

In [ ]:
def read_source(dir_source):
    file1 = open(dir_source, 'r')
    Lines = file1.readlines()
    x=np.float(Lines[2].split('=')[1].split('#')[0])
    z=np.float(Lines[3].split('=')[1].split('#')[0])
    return x,z
def read_receiver(dir_receiver):
    import re
    file1 = open(dir_receiver, 'r')
    Lines = file1.readlines()
    number=len(Lines)
    receiver_station=[]
    receiver_net=[]
    receiver_x=np.zeros(number)
    receiver_z=np.zeros(number)
    for _i, line in enumerate(Lines):
        station,network,x,z,nun,nun=re.findall(r'\S+', line)
        #print(station)
        receiver_station.append(station)
        receiver_net.append(network)
        receiver_x[_i]=np.float(x)
        receiver_z[_i]=np.float(z)
    return receiver_station,receiver_net,receiver_x,receiver_z

def launch_adjoint(dir_run,dir_event,dir_storesyn,modelfile):
    from datetime import datetime
    os.chdir(dir_run)
    os.chdir(dir_event)
    from shutil import copyfile
    import shutil
    from distutils.dir_util import copy_tree
    #copyfile(modelfile,'DATA/profile.xyz')
    #os.remove('change_simulation_type.pl')
    #s.system('ln -s /home/yjgao/install/specfem2d-devel/utils/change_simulation_type.pl')
    #s.system('perl change_simulation_type.pl -f') 
    copyfile(modelfile,'DATA/profile.xyz')
    copyfile('../run_this_example_kernel.sh','run_this_example_kernel.sh')
    print('start adjoint modelling',dir_event)
    os.system('bash run_this_example_kernel.sh')
    print('finished adjoint modelling')
    ###store the synthetics to the dir observed or syn for backup
    #!cat OUTPUT_FILES/proc000*_rho_kappa_mu_kernel.dat > OUTPUT_FILES/rho_kappa_mu_kernel.dat
    ####prepared to 
    os.system('cat OUTPUT_FILES/proc000*_rhop_alpha_beta_kernel.dat > OUTPUT_FILES/rho_alpha_beta_kernel.dat')
    shutil.copy('OUTPUT_FILES/rho_alpha_beta_kernel.dat', dir_storesyn)
    os.chdir(dir_run)
def adj_calculate(dir_run,dir_event,dir_obs,dir_syn,adj_src_type,stationpath):
    import obspy
    import pyadjoint
    import numpy as np

    from obspy.taup import TauPyModel
    from obspy.core.trace import Trace
    ##guarantee to change back to the project directory
    os.chdir(dir_run)
    os.chdir(dir_event)
    ##### read source locations
    source_x,source_depth_in_m=read_source('DATA/SOURCE')
    ##### read receiver locations
    receiver_station,receiver_net,receiver_x,receiver_z=read_receiver(stationpath)
    import toml
    Misfits_all=0.0
    fh = open(dir_syn+'misfit.txt','w')
    for i, station_name in enumerate(receiver_station):
        #print(i)
        #print(np.int(i))
        distance=np.abs(receiver_x[i]-source_x)/111.0/1000.0
        model = TauPyModel(model="iasp91")
        # adapted 
        arrivals = model.get_travel_times(source_depth_in_km=-source_depth_in_m/1000,distance_in_degree=distance) 
        time=arrivals[0].time
        if time > 400-15 :
            window=[]
        else:
            window=[[time-3,time+14]]
        print(window)
        network=receiver_net[i]
        station=receiver_station[i]
        data_hetero=np.loadtxt(dir_syn+network+'.'+station+'.BXZ'+'.semd')
        data_hetero_new=data_hetero.swapaxes(0,1)
    #from obspy.core.trace import Trace
        tr=Trace(data=data_hetero_new[1])
        tr.stats.delta=0.02  ###here should be adapted to the DATA/Par_file
        tr.stats.network = network
        tr.stats.station = station
        tr.stats.channel = 'BXZ'
        data_hetero_obs=np.loadtxt(dir_obs+network+'.'+station+'.BXZ'+'.semd')                           
        data_hetero_new_obs=data_hetero_obs.swapaxes(0,1)
        tr_obs=Trace(data=data_hetero_new_obs[1])
        tr_obs.stats.delta=0.02
        tr_obs.stats.network = network
        tr_obs.stats.station = station
        tr_obs.stats.channel = 'BXZ'
        ###here input a minum and maximum period
        configure=pyadjoint.config.Config(4,100,measure_type='dt')
        print(configure)
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=4, max_period=100,
        config=configure, window=window, plot=True);
        Misfits_all+=adj.misfit
        adj.write(filename='SEM/'+network+'.'+station+'.BXZ'+'.adj',format="SPECFEM", time_offset=-6)
        
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=4, max_period=100,
        config=configure, window=[], plot=True);
        adj.write(filename='SEM/'+network+'.'+station+'.BXY'+'.adj',format="SPECFEM", time_offset=-6)
        
        data_hetero=np.loadtxt(dir_syn+network+'.'+station+'.BXX'+'.semd')
        data_hetero_new=data_hetero.swapaxes(0,1)
        tr=Trace(data=data_hetero_new[1])
        tr.stats.delta=0.02
        tr.stats.network = network
        tr.stats.station = station
        tr.stats.channel = 'BXX'
        data_hetero_obs=np.loadtxt(dir_obs+network+'.'+station+'.BXX' +'.semd')                           
        data_hetero_new_obs=data_hetero_obs.swapaxes(0,1)
        tr_obs=Trace(data=data_hetero_new_obs[1])
        tr_obs.stats.delta=0.02
        tr_obs.stats.network = network
        tr_obs.stats.station = station
        tr_obs.stats.channel = 'BXX'
        adj=pyadjoint.calculate_adjoint_source(
        adj_src_type=adj_src_type, observed=tr_obs, synthetic=tr, min_period=4, max_period=100,
        config=configure, window=window, plot=True);
        print(adj)
        adj.write(filename='SEM/'+network+'.'+station+'.BXX'+'.adj',format="SPECFEM", time_offset=-6)
    fh.write(dir_syn+' ' + dir_event +' ' + str(Misfits_all)+"\n")
    fh.close()
    os.chdir(dir_run)

In [ ]:
list_events=['EVENT12','EVENT13']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','tf_phase_misfit','DATA/STATIONS')

In [ ]:
list_events=['EVENT12','EVENT13']
for _i, event in enumerate(list_events):
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)

In [ ]:

import os.path
import sys
import numpy as np

try:
    import matplotlib.pyplot as plt
except:
    print("Error importing pyplot from matplotlib, please install matplotlib package first...")
    sys.tracebacklimit=0
    raise Exception("Importing matplotlib failed")
###controlling the speed for smoothing!
def grid(x, y, z, resX=200, resY=200):
    """
    Converts 3 column data to matplotlib grid
    """
    from scipy.interpolate import griddata
    #from scipy.interpolate import griddata

    xi = np.linspace(min(x), max(x), resX)
    yi = np.linspace(min(y), max(y), resY)

    # mlab version
    Z = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')
    # scipy version
    #Z = griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

    X, Y = np.meshgrid(xi, yi)
    return X, Y, Z

def plot_kernels(filename,show=False,EVENT='EVENT',total_max=1e-10,figurename='test.png'):
    """
    plots ASCII kernel file
    """
    #print "plotting kernel file: ",filename
    #print ""

    data = np.loadtxt(filename)

    # checks data
    if data.ndim != 2:
        #print "Error: wrong data dimension for kernel file",data.ndim
        sys.tracebacklimit=0
        raise Exception("Invalid data dimension")

    # checks array
    if len(data[1,:]) != 5:
        #print "data shape  : ",data.shape
        #print "data lengths: ",len(data[:,1]),len(data[1,:])
        #print "Error: wrong data format for kernel file",data.shape
        sys.tracebacklimit=0
        raise Exception("Invalid data format")

    # splits up data
    x = data[:,0]
    y = data[:,1]

    #print "dimensions:"
    #print "  x-range min/max = %f / %f" % (x.min(), x.max())
    #print "  y-range min/max = %f / %f" % (y.min(), y.max())
    #print ""

    z1 = data[:,2] # e.g. rho
    z2 = data[:,3] # e.g. alpha
    z3 = data[:,4] # e.g. beta

    # names like
    #   rhop_alpha_beta_kernel.dat
    # or
    #   proc000000_rhop_alpha_beta_kernel.dat
    name = os.path.basename(filename)

    name_kernels = str.split(name,"_")
    if len(name_kernels) == 4:
        kernel1 = 'K_' + name_kernels[0] # rhop
        kernel2 = 'K_' + name_kernels[1] # alpha
        kernel3 = 'K_' + name_kernels[2] # beta
    elif len(name_kernels) == 5:
        kernel1 = 'K_' + name_kernels[1]
        kernel2 = 'K_' + name_kernels[2]
        kernel3 = 'K_' + name_kernels[3]
    else:
        kernel1 = 'K_1'
        kernel2 = 'K_2'
        kernel3 = 'K_3'

    #print "statistics:"
    #print "  %12s : min/max = %e / %e" % (kernel1,z1.min(),z1.max())
    #print "  %12s : min/max = %e / %e" % (kernel2,z2.min(),z2.max())
    #print "  %12s : min/max = %e / %e" % (kernel3,z3.min(),z3.max())
    #print ""

    #total_max = 0.1*abs(z2).max()
    print(total_max)
    #print "  data max = ",total_max
    #print ""
    #print(total_max)
    #total_max = 1.e-8

    # setup figure (with 3 subplots)
    fig, axes = plt.subplots(nrows=3, ncols=1,figsize=(10,10))

    for i,ax in enumerate(axes.flat,start=1):
        # top
        if i == 1:
            X, Y, Z = grid(x,y,z1)
            ax.set_title("Kernels")
            ax.set_ylabel('RHO')
        elif i == 2:
            X, Y, Z = grid(x,y,z2)
            ax.set_ylabel('VP')
        elif i == 3:
            X, Y, Z = grid(x,y,z3)
            ax.set_ylabel('VS')

        #colormap = 'jet'
        colormap = 'RdBu'
        print(np.max(np.abs(Z)))
        im = ax.imshow(Z/np.max(np.abs(Z)),vmax=total_max, vmin=-total_max,extent=[0,25*111,0,800],cmap=colormap)
        ax.plot([11*111,14*111,14*111,11*111,11*111],[140,140,230,230,140],'black')
        ax.plot(666,121,'o',color='black') #1
        ax.plot(666,221,'o',color='black') #2
        ax.plot(666,321,'o',color='black') #3
        ax.plot(1400,321,'o',color='black') #4
        ax.plot(2200,121,'o',color='black') #5 
        ax.plot(2200,221,'o',color='black') #6 
        ax.plot(2200,321,'o',color='black') #7
        ax.plot(1100,321,'o',color='black') #8
        ax.plot(1700,321,'o',color='black') #9
        ax.plot(1100,221,'o',color='black') #10
        ax.plot(1700,221,'o',color='black') #11
        ax.plot(1100,121,'o',color='black') #13
        ax.plot(1700,121,'o',color='black') #12
               # im = ax.imshow(Z,vmax=total_max, vmin=total_min,extent=[0,45*111,0,1000],cmap=colormap.reversed())
      #  ax.plot([18*111,25*111,25*111,18*111,18*111],[500,500,660,660,500],'black')
      #  ax.plot([0,45*111],[410,410],'black')
      #  ax.plot([0,45*111],[660,660],'black')
      #  ax.text(10,390,'410 km', fontsize=9)
      #  ax.text(10,640,'660 km', fontsize=9)
        ax.invert_yaxis()
        #ax.set_xlim(2000000,3000000)
        #ax.set_ylim(-200000,0)
    # moves plots together
    fig.subplots_adjust(hspace=0)
    #plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)
    
    # colorbar
    fig.colorbar(im, ax=axes.ravel().tolist())
    #plt.xlim(2500000,3000000)
    #plt.ylim(-200000,0)
    #fig.colorbar(im, ax=axes.ravel().tolist(),orientation='horizontal')

    # show the figure
    if show:
        plt.show()

    # saves kernel figure as file
    dir = os.path.dirname('.')
    name_without_ending = str.split(name,".")[0]
    outfile = dir + "/" + name_without_ending + ".png"
    fig.savefig(figurename, format="png",  dpi=300)

    #print "*****"
    #print "plotted file: ",outfile
    #print "*****"
    #print ""

In [ ]:
def read_gradient(dir_gradient):
    gradient=np.loadtxt(dir_gradient)
    x=gradient.T[0]
    z=gradient.T[1]
    rho=gradient.T[2]
    alpha=gradient.T[3]
    belta=gradient.T[4]
    return x,z,rho,alpha,belta
def read_modelfile(modelfile):
    model=np.loadtxt('MODELS/'+modelfile)
    x_new=model.T[0]
    z_new=model.T[1]
    vp=model.T[2]
    vs=model.T[3]
    rho=model.T[4]
    return x_new,z_new,vp,vs,rho
def sum_gradient(events,iteration,smoothsrc='True'):
    if smoothsrc=='True':
        gradient=np.loadtxt(list_events[0]+'/REF_SEM_'+iteration+'/'+'rho_alpha_beta_kernelsmsrc.dat')
        pendix='rho_alpha_beta_kernelsmsrc.dat'
    else:
        gradient=np.loadtxt(list_events[0]+'/REF_SEM_'+iteration+'/'+'rho_alpha_beta_kernel.dat')
        pendix='rho_alpha_beta_kernel.dat'
    gradient_sum=np.zeros((gradient.shape))
    for _i,gradient in enumerate(list_events):
        x,z,rho,alpha,belta=read_gradient(list_events[_i]+'/REF_SEM_'+iteration+'/'+pendix)
        gradient_sum.T[2]+=rho
        gradient_sum.T[3]+=alpha
        gradient_sum.T[4]+=belta
    gradient_sum.T[0]=x
    gradient_sum.T[1]=z
    np.savetxt(iteration+'_summed_gradient.dat',gradient_sum)
    return gradient
def cut_source_region_from_gradient_gauss(EVENT:str,iteration:str, radius_to_cut: float):
    """
    Sources often show unreasonable sensitivities. This function
    brings the value of the gradient down to zero for that region.
    I recommend doing this before smoothing.
    
    :param mesh: Path to the mesh
    :type mesh: str
    :param source_location: Source latitude, longitude and depth
    :type source_location: dict
    :param radius_to_cut: Radius to cut in km
    :type radius_to_cut: float
    """
    import lasif
    dir_gradient= EVENT+ '/REF_SEM_' +iteration+'/rho_alpha_beta_kernel.dat'
    #print(mesh)
    x,z,rho,alpha,belta=read_gradient(dir_gradient)
    gradient=np.loadtxt(dir_gradient)
    s_x,s_z=read_source(EVENT+'/DATA/SOURCE')
    dist = np.sqrt( (x - s_x) ** 2 + (z - s_z) ** 2)
    
    ####here we implement 1-Gaussian smoothing the source
    #distance_smooth = (x[k] * np.ones(number_nodes) - x) ** 2 + (y[k] * np.ones(number_nodes) - y)** 2 + (z[k] * np.ones(number_nodes) - z) ** 2
    distance_smooth1 = np.exp(-dist/ (2 * radius_to_cut ** 2))
    print(dist.max(),distance_smooth1.max())
    distance_smooth1 = distance_smooth1 / distance_smooth1.max()
    print(distance_smooth1.max())
    alpha_new=alpha*(1-distance_smooth1)
    belta_new=belta*(1-distance_smooth1)
    rho_new=rho*(1-distance_smooth1)
    gradient_new=gradient.copy()
    smooth_dist=gradient.copy()
    gradient_new.T[2]=rho_new
    gradient_new.T[3]=alpha_new
    gradient_new.T[4]=belta_new
    smooth_dist.T[2]=1-distance_smooth1
    smooth_dist.T[3]=1-distance_smooth1
    smooth_dist.T[4]=1-distance_smooth1
    np.savetxt(EVENT+ '/REF_SEM_' +iteration+'/rho_alpha_beta_kernelsmsrc.dat',gradient_new)
    np.savetxt(EVENT+ '/REF_SEM_' +iteration+'/rho_alpha_beta_kernelsdist.dat',smooth_dist)
    


def smooth_cupy_new(xyz_matrix,iterations,sigmaxy,sigmaz,Velocity_matrix): 
      ##input a numpy array and using asarray move to the gpu device
      import cupy as cp
      Velocity_matrix=cp.asarray(Velocity_matrix)
      xyz_matrix=cp.asarray(xyz_matrix)
      Vpv_smooth_all=cp.zeros((3,iterations),dtype=float)  
      #t1 = datetime.datetime.now()       
      for i in range(iterations):  
            distancex=(cp.square(xyz_matrix[0][i]-xyz_matrix[0]))/(2*sigmaxy*sigmaxy)
            distancey=(cp.square(xyz_matrix[1][i]-xyz_matrix[1]))/(2*sigmaxy*sigmaxy)
            #distancez=(cp.square(xyz_matrix[2][i]-xyz_matrix[2]))/(2*sigmaz*sigmaz)
            #print(distance)
            distance=distancex+distancey
            distance=cp.exp(-distance)
            distance= cp.divide(distance,cp.sum(distance))
            #print(distance)
            Vpv_smooth_all[0][i] = cp.dot(distance, Velocity_matrix[0])
            Vpv_smooth_all[1][i] = cp.dot(distance, Velocity_matrix[1])
            Vpv_smooth_all[2][i] = cp.dot(distance, Velocity_matrix[2])
      #print(t2-t1)
      # move array from gpu device to the host
      return cp.asnumpy(Vpv_smooth_all)

def smooth_numpy_new(xyz_matrix,iterations,sigmaxy,sigmaz,Velocity_matrix): 
      ##input a numpy array and using asarray move to the gpu device
     # import cupy as cp
      Velocity_matrix=np.asarray(Velocity_matrix)
      xyz_matrix=np.asarray(xyz_matrix)
      Vpv_smooth_all=np.zeros((3,iterations),dtype=float)  
      #t1 = datetime.datetime.now()       
      for i in range(iterations):  
            distancex=(np.square(xyz_matrix[0][i]-xyz_matrix[0]))/(2*sigmaxy*sigmaxy)
            distancey=(np.square(xyz_matrix[1][i]-xyz_matrix[1]))/(2*sigmaxy*sigmaxy)
            #distancez=(cp.square(xyz_matrix[2][i]-xyz_matrix[2]))/(2*sigmaz*sigmaz)
            #print(distance)
            distance=distancex+distancey
            distance=np.exp(-distance)
            distance= np.divide(distance,np.sum(distance))
            #print(distance)
            Vpv_smooth_all[0][i] = np.dot(distance, Velocity_matrix[0])
            Vpv_smooth_all[1][i] = np.dot(distance, Velocity_matrix[1])
            Vpv_smooth_all[2][i] = np.dot(distance, Velocity_matrix[2])
      #print(t2-t1)
      # move array from gpu device to the host
      return Vpv_smooth_all


def smooth_compat(x,y,Vp,Vs,rho,sigmaxy,sigmaz):   
    number_nodes=x.shape[0]
    print(number_nodes)
    V_smooth_all=np.zeros((3,number_nodes),dtype=float)
    xyz_matrix=np.zeros((2,number_nodes),dtype=float)
    xyz_matrix[0]=x
    xyz_matrix[1]=y
    Velocity_matrix=np.zeros((3,number_nodes),dtype=float)
    Velocity_matrix[0]=Vp
    Velocity_matrix[1]=Vs
    Velocity_matrix[2]=rho
    iterations=number_nodes
    #Vpv1_smooth_all=np.zeros((5,206313),dtype=float)
    test_cupy=smooth_cupy_new(xyz_matrix,iterations,sigmaxy,sigmaz,Velocity_matrix)
    return test_cupy

def smooth_kernel(gradient,iteration,sigmax,sigmaz):
    x,z,rho_kernel,alpha_kernel,belta_kernel=read_gradient(gradient)
    smooth_format=smooth_compat(x,z,alpha_kernel,belta_kernel,rho_kernel,sigmax,sigmaz)
    smooth_kernel=np.zeros((5,x.shape[0]))
    smooth_kernel[0]=x
    smooth_kernel[1]=z
    ####follow the sequence of specfem2d for the gradient field rho, p, s
    smooth_kernel[2]=smooth_format[2]
    smooth_kernel[3]=smooth_format[0]
    smooth_kernel[4]=smooth_format[1]
    np.savetxt(iteration+'_summed_gradient_smooth.dat',smooth_kernel.T)

    
###since most of the artifacts around the source are removed, the remainings are derived from the surface beneath the receivers
### Because remove the artifacts around receivers are much more expensive, it would be great to desighn a function to suppress the 
### gradients with depth
def remove_artifacts(gradient, iteration, supress_min, supress_depth):
    x,z,rho_kernel,alpha_kernel,belta_kernel=read_gradient(gradient)
    suppress=1-np.exp(-z**2/ (2 * supress_depth ** 2))+supress_min
    rho_kernel_suppress=suppress*rho_kernel
    alpha_kernel_suppress=suppress*alpha_kernel
    belta_kernel_suppress=suppress* belta_kernel
    supress_kernel=np.zeros((5,x.shape[0]))
    supress_kernel[0]=x
    supress_kernel[1]=z
    ####follow the sequence of specfem2d for the gradient field rho, p, s
    supress_kernel[2]=rho_kernel_suppress
    supress_kernel[3]=alpha_kernel_suppress
    supress_kernel[4]=belta_kernel_suppress
    plt.plot(z,suppress,'o')
    np.savetxt(iteration+'_summed_gradient_supress.dat',supress_kernel.T)
def interpolate_kernel_format(gradient,x_new,z_new):
    x,z,rho_kernel,alpha_kernel,belta_kernel=read_gradient(gradient)
    from scipy.interpolate import griddata
    alpha_kernel_newformat=griddata((x,z),alpha_kernel,(x_new,z_new),method='cubic')
    belta_kernel_newformat=griddata((x,z),belta_kernel,(x_new,z_new),method='cubic')
    rho_kernel_newformat=griddata((x,z),rho_kernel,(x_new,z_new),method='cubic')
###here because of the nan when using cubic at the boundary points
    nan_index=np.argwhere(np.isnan(alpha_kernel_newformat))
    alpha_kernel_newformat[nan_index]=griddata((x,z),alpha_kernel,(x_new[nan_index],z_new[nan_index]),method='nearest')
    belta_kernel_newformat[nan_index]=griddata((x,z),belta_kernel,(x_new[nan_index],z_new[nan_index]),method='nearest')
    rho_kernel_newformat[nan_index]  =griddata((x,z),rho_kernel,  (x_new[nan_index],z_new[nan_index]),method='nearest')
    gradient_new=np.zeros((3,x_new.shape[0]))
    gradient_new[2]=rho_kernel_newformat
    gradient_new[0]=alpha_kernel_newformat
    gradient_new[1]=belta_kernel_newformat
    
    return gradient_new
def model_update_steepest(gradient,model,perturb_Vp,perturb_Vs,perturb_rho):
    model=np.loadtxt(model)
    x_new=model.T[0]
    z_new=model.T[1]
    vp=model.T[2]
    vs=model.T[3]
    rho=model.T[4]
    ##through this, reforge the shape from rho, vp and vs to vp vs and rho, and interpolate
    gradient=interpolate_kernel_format(gradient,x_new,z_new)
    gradient_Vp_abs=np.absolute(gradient[0]) 
    gradient_Vs_abs=np.absolute(gradient[1]) 
    gradient_rho_abs=np.absolute(gradient[2]) 

    print('maxgrdientVp','maxgrdientVs','maxgrdientrho')
    print(np.max(gradient_Vp_abs),np.max(gradient_Vs_abs),np.max(gradient_rho_abs))  
    alpha_Vp=perturb_Vp*vp/gradient_Vp_abs 
    alpha_Vs=perturb_Vs*vs/gradient_Vs_abs 
    alpha_rho=perturb_rho*rho/gradient_rho_abs
    step_Vp= np.min(alpha_Vp)
    step_Vs= np.min(alpha_Vs)
    step_rho= np.min(alpha_rho)
    print('stepVp','stepVs','steprho')
    print( step_Vp,step_Vs,step_rho)
    print('variationmax for Vp,  Vs, Vsh,rho')
    print( np.max(np.abs(step_Vp*gradient[0])),np.max(np.abs(step_Vs*gradient[1])),np.max(np.abs(step_rho*gradient[2])))
    VpNew= model.T[2] - step_Vp*gradient[0]
    VsNew= model.T[3] - step_Vs*gradient[1]
    rhoNew= model.T[4] - step_rho*gradient[2]
    return x_new,z_new,VpNew,VsNew,rhoNew

def generate_newmodel(vp,vs,rho,x,z,newfile,newfile_noheader):  
    import numpy as np
    import matplotlib.pyplot as plt
    maxx=45*111*1000
    minx=0
    maxz=0
    minz=-1000*1000
    nx = 800 # Number of sampling points in x direction
    nz = 200  # Number of sampling points in z direction
    spacing_x = (maxx-minx)/(nx-1)
    spacing_z = (maxz-minz)/(nz-1)
    print ("space x space z", spacing_x,spacing_z)
    fo = open(newfile, "w+") # Name of the file it has to be set in the Par_file
    fo2 = open(newfile_noheader, "w+")
    print( "Name of the file: ", fo.name)
    orig_x=0
    orig_z=0
    end_x=maxx
    end_z=maxz
    line1 = str(orig_x)+" "+str(orig_z)+" "+str(end_x)+" "+str(end_z)+"\n"
    line2 = str(spacing_x)+" "+str(spacing_z)+"\n"
    line3 = str(nx)+" "+str(nz)+"\n"
# line4 : vpMin vpMax vsMin vsMax rhoMin rhoMax
    line4 = str(min(vp))+" "+str(max(vp))+" "+str(min(vs))+" "+str(max(vs))+" "+str(min(rho))+" "+str(max(rho))+"\n"
# Write a line at the end of the file.
    fo.write(line1)
    fo.write(line2)
    fo.write(line3)
    fo.write(line4)
    m=x.shape[0]
    for i in np.arange(0,m):
        lineToWrite= str(x[i])+" "+str(z[i])+" "+str(vp[i])+" "+str(vs[i])+" "+str(rho[i])+"\n"
        fo.write(lineToWrite)
        fo2.write(lineToWrite)
    fo.close()
    fo.close()
def retrieve_misfit(list_events,ITERATION):
    import re
    ALL_misfit=0
    for _i, event in enumerate(list_events):
        print(event)
        file1 = open(event+'/'+'REF_SEM_'+ITERATION+'/misfit.txt', 'r')
        Lines = file1.readlines()
        path,event,misfit=re.findall(r'\S+', Lines[0])
        print(misfit)
        ALL_misfit+=float(misfit)
    return ALL_misfit
        
def assemble_lbfgs(listofgradients,listofmodels):
    import h5py
    from lasif import function_store
    from scipy.interpolate import griddata
    x,z,rho,alpha,belta=read_gradient(listofgradients[0])
    x_new,z_new,vp,vs,rho=read_modelfile(listofmodels[0])
    fields=['vp','vs','rho']
    ###becareful the shape of model and gradient is differen
    g=np.zeros((len(listofgradients),3,x_new.shape[0]))
    m=np.zeros((len(listofgradients),3,x_new.shape[0]))
    modelupdate=np.zeros((3,x_new.shape[0]))
    for i in np.arange(len(listofgradients)):
        gradient=interpolate_kernel_format(listofgradients[i],x_new,z_new)
        x_new,z_new,vp,vs,rho =read_modelfile(listofmodels[i])
        model=np.zeros((3,x_new.shape[0]))
        model[0]=vp
        model[1]=vs 
        model[2]=rho
        g[i]=gradient
        m[i]=model
    g=np.swapaxes(g,0,1)
    m=np.swapaxes(m,0,1)
    
    ##swap axes for loop the fields
    for _i, field in enumerate(fields):
        modelupdate[_i]=function_store.LBFGS_n_order(g[_i],m[_i])
    return modelupdate
def model_update_lbfgs(listofgradients,listofmodels,percent_vp,percent_vs,percent_rho,smoothfactor_x,smoothfactor_z,newfile,newfile_noheader):
    model_update=assemble_lbfgs(listofgradients,listofmodels)
    print('finish lbfgs')
    x_new,z_new,vp,vs,rho=read_modelfile(listofmodels[-1])
    smooth_format=smooth_compat(x_new,z_new,model_update[0],model_update[1],model_update[2],smoothfactor_x,smoothfactor_z)
    print('finish smoothing')
    vp_new=vp-percent_vp*smooth_format[0]
    vs_new=vs-percent_vs*smooth_format[1]
    rho_new=rho-percent_rho*smooth_format[2]
    print(np.max(np.abs(percent_vp*smooth_format[0])),np.max(percent_vs*smooth_format[1]),np.max(percent_rho*smooth_format[2]))
    generate_newmodel(vp_new,vs_new,rho_new,x_new,z_new,newfile,newfile_noheader)
    

In [ ]:
print(read_source('EVENT1/DATA/SOURCE'))
print(read_source('EVENT2/DATA/SOURCE'))
print(read_source('EVENT3/DATA/SOURCE'))
print(read_source('EVENT4/DATA/SOURCE'))
print(read_source('EVENT5/DATA/SOURCE'))
print(read_source('EVENT6/DATA/SOURCE'))
print(read_source('EVENT7/DATA/SOURCE'))
print(read_source('EVENT8/DATA/SOURCE'))
print(read_source('EVENT9/DATA/SOURCE'))
print(read_source('EVENT10/DATA/SOURCE'))
print(read_source('EVENT11/DATA/SOURCE'))
print(read_source('EVENT12/DATA/SOURCE'))
print(read_source('EVENT13/DATA/SOURCE'))

In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
iteration='smooth'
for _i,event in enumerate(list_events):
    cut_source_region_from_gradient_gauss(event,iteration, 200)
sum_gradient(list_events,iteration, smoothsrc='True')

In [ ]:
plot_kernels('smooth_summed_gradient.dat',show=False,EVENT='summed_smooth',total_max=0.5,figurename='summed.png')

In [ ]:
!cp smooth_summed_gradient.dat smooth_summed_gradient_tf.dat

In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','ccc','DATA/STATIONS')
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)

In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
iteration='smooth'
for _i,event in enumerate(list_events):
    cut_source_region_from_gradient_gauss(event,iteration, 200)
sum_gradient(list_events,iteration, smoothsrc='True')

In [ ]:
!cp smooth_summed_gradient.dat smooth_summed_gradient_ccc.dat

In [ ]:
plot_kernels('smooth_summed_gradient_ccc.dat',show=False,EVENT='summed_smooth',total_max=1,figurename='summed_ccc.png')

In [ ]:
plot_kernels('smooth_summed_gradient_tf.dat',show=False,EVENT='summed_smooth',total_max=0.5,figurename='summed_tf.png')

In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','waveform_misfit','DATA/STATIONS')
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)

list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
iteration='smooth'
for _i,event in enumerate(list_events):
    cut_source_region_from_gradient_gauss(event,iteration, 200)
sum_gradient(list_events,iteration, smoothsrc='True')

In [ ]:
!cp smooth_summed_gradient.dat smooth_summed_gradient_l2.dat

In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','cc_traveltime_misfit','DATA/STATIONS')
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)



In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
iteration='smooth'
for _i,event in enumerate(list_events):
    cut_source_region_from_gradient_gauss(event,iteration, 200)
sum_gradient(list_events,iteration, smoothsrc='True')

In [ ]:
!cp smooth_summed_gradient.dat smooth_summed_gradient_cc.dat

In [ ]:
#list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
list_events=['EVENT8']
modelfile=PROJECT+'MODELS/profile_obs_smooth.xyz'
print('tf')
for _i, event in enumerate(list_events):
    adj_calculate(PROJECT,event,'REF_SEM/','REF_SEM_smooth/','multitaper_misfit','DATA/STATIONS')
    launch_adjoint(PROJECT,event,'REF_SEM_smooth/',modelfile)



In [ ]:
list_events=['EVENT5','EVENT1','EVENT2','EVENT3','EVENT4','EVENT6','EVENT7','EVENT8','EVENT9','EVENT10','EVENT11','EVENT12','EVENT13']
iteration='smooth'
for _i,event in enumerate(list_events):
    cut_source_region_from_gradient_gauss(event,iteration, 200)
sum_gradient(list_events,iteration, smoothsrc='True')

In [ ]:
!cp smooth_summed_gradient.dat smooth_summed_gradient_mt.dat

In [ ]:
def extractkernelprofile(kernelfile):
    data = np.loadtxt(kernelfile)
    x = data[:,0]
    y = data[:,1]
    z2 = data[:,3] # e.g. alpha
    X, Y, Z = grid(x,y,z2)
    newX=X.swapaxes(0,1)
    newZ=Z.swapaxes(0,1)
    newY=Y.swapaxes(0,1)
    return newX,newY,newZ

In [ ]:
newX,newY,newZ=extractkernelprofile('smooth_summed_gradient_l2.dat')
newX,newY,newZmt=extractkernelprofile('smooth_summed_gradient_mt.dat')
newX,newY,newZcc=extractkernelprofile('smooth_summed_gradient_cc.dat')
newX,newY,newZccc=extractkernelprofile('smooth_summed_gradient_ccc.dat')
newX,newY,newZtf=extractkernelprofile('smooth_summed_gradient_tf.dat')

In [ ]:
data = np.loadtxt('smooth_summed_gradient_mt.dat')
x = data[:,0]
y = data[:,1]
z2 = data[:,3] # e.g. alpha
Xmt, Ymt, Zmt = grid(x,y,z2)

In [ ]:
newXmt=Xmt.swapaxes(0,1)
newZmt=Zmt.swapaxes(0,1)
newYmt=Ymt.swapaxes(0,1)

In [ ]:
newYmt.T[200-73]/1000

In [ ]:
plt.plot(newXmt.T[200-50],newZmt.T[200-50])

In [ ]:
modelfile_smooth='MODELS/profile_obs_numpy_smooth.xyz'
modelfile='MODELS/profile_obs_numpy.xyz'
model_smooth=np.loadtxt(modelfile_smooth)
model=np.loadtxt(modelfile)
diff=model_smooth.T[2]-model.T[2]
y_diff=model.T[1].reshape(200,720).swapaxes(0,1)[301]
m_diff=diff.reshape(200,720).swapaxes(0,1)[301]

In [ ]:
model.T[1].reshape(200,720).swapaxes(0,1).T[200-62]/1000

In [ ]:
plt.figure(figsize=[16, 18])
depth=np.linspace(900,0,len(y_diff))
depth_kernel=np.linspace(800,0,len(newY[100]))
modelfile_smooth='MODELS/profile_obs_numpy_smooth.xyz'
modelfile='MODELS/profile_obs_numpy.xyz'
model_smooth=np.loadtxt(modelfile_smooth)
model=np.loadtxt(modelfile)
diff=model_smooth.T[2]-model.T[2]
y_diff=model.T[1].reshape(200,720).swapaxes(0,1)[301]
m_diff=diff.reshape(200,720).swapaxes(0,1)[301]

plt.subplot(411)
plt.plot(depth,m_diff/m_diff.max(),'k',linewidth=10,alpha=0.5,label='model diff.')
plt.plot(depth_kernel,newZ[100]/newZ[100].max(),'r',label='L2')
plt.plot(depth_kernel,newZcc[100]/newZcc[100].max(),'b',label='CC time shift')
plt.plot(depth_kernel,newZccc[100]/newZccc[100].max(),'g',label='CCC')
plt.plot(depth_kernel,newZtf[100]/newZtf[100].max(),'m',label='TF phase shift')
plt.plot(depth_kernel,newZmt[100]/newZmt[100].max(),'k',label='multitaper time shitf')
plt.text(10,0.9,'(a)')
plt.legend()
plt.xlabel('Depth (km)')
plt.ylabel('Normalised Kernel (P wave)')
plt.xlim(0,400)



plt.subplot(412)
y_diff=model.T[1].reshape(200,720).swapaxes(0,1).T[174]
m_diff=diff.reshape(200,720).swapaxes(0,1).T[174]
#depth_kernel=np.linspace(0,30*111*1000,len(newX[100]))
depth=np.linspace(0,111*30*1000,len(y_diff))
plt.plot(depth/1000,m_diff/m_diff.max(),'k',linewidth=10,alpha=0.5)
plt.plot(newX.T[200-29]/1000,newZmt.T[200-29]/newZmt.T[200-29].max(),'k')
plt.plot(newX.T[200-29]/1000,newZcc.T[200-29]/newZcc.T[200-29].max(),'b')
plt.plot(newX.T[200-29]/1000,newZccc.T[200-29]/newZccc.T[200-29].max(),'g')
plt.plot(newX.T[200-29]/1000,newZ.T[200-29]/newZ.T[200-29].max(),'r')
plt.plot(newX.T[200-29]/1000,newZtf.T[200-29]/newZtf.T[200-29].max(),'m')
plt.text(100,0.9,'(b)')
plt.xlabel('X (km)')
plt.ylabel('Normalised Kernel (P wave)')
plt.xlim(0,25*111)

plt.subplot(413)
y_diff=model.T[1].reshape(200,720).swapaxes(0,1).T[200-51]
m_diff=diff.reshape(200,720).swapaxes(0,1).T[200-51]
#depth_kernel=np.linspace(0,30*111*1000,len(newX[100]))
depth=np.linspace(0,111*30*1000,len(y_diff))
plt.plot(depth/1000,m_diff/m_diff.min(),'k',linewidth=10,alpha=0.5)
plt.plot(newX.T[200-57]/1000,newZmt.T[200-57]/newZmt.T[200-57].min(),'k')
plt.plot(newX.T[200-57]/1000,newZcc.T[200-57]/newZcc.T[200-57].min(),'b')
plt.plot(newX.T[200-57]/1000,newZccc.T[200-57]/newZccc.T[200-57].min(),'g')
plt.plot(newX.T[200-57]/1000,newZ.T[200-57]/newZ.T[200-57].min(),'r')
plt.plot(newX.T[200-57]/1000,newZtf.T[200-57]/newZtf.T[200-57].min(),'m')
plt.text(100,0.9,'(c)')
plt.xlabel('X (km)')
plt.ylabel('Normalised Kernel (P wave)')
#plt.plot(depth,m_diff/m_diff.max(),'k',linewidth=10,alpha=0.5,label='model diff.')
#plt.plot(depth_kernel,newZ[100]/newZ[100].max(),'r',label='L2')
#plt.plot(depth_kernel,newZcc[100]/newZcc[100].max(),'b',label='CC time shift')
#plt.plot(depth_kernel,newZccc[100]/newZccc[100].max(),'g',label='CCC')
#plt.plot(depth_kernel,newZtf[100]/newZtf[100].max(),'m',label='TF phase shift')
#plt.plot(depth_kernel,newZmt[100]/newZmt[100].max(),'k',label='multitaper time shitf')
plt.legend()
plt.xlim(0,25*111)

plt.subplot(414)
y_diff=model.T[1].reshape(200,720).swapaxes(0,1).T[200-62]
m_diff=diff.reshape(200,720).swapaxes(0,1).T[200-62]
#depth_kernel=np.linspace(0,30*111*1000,len(newX[100]))
depth=np.linspace(0,111*30*1000,len(y_diff))
plt.plot(depth/1000,m_diff/m_diff.max(),'k',linewidth=10,alpha=0.5)
plt.plot(newX.T[200-69]/1000,newZmt.T[200-69]/newZmt.T[200-69].max(),'k')
plt.plot(newX.T[200-69]/1000,newZcc.T[200-69]/newZcc.T[200-69].max(),'b')
plt.plot(newX.T[200-69]/1000,newZccc.T[200-69]/newZccc.T[200-69].max(),'g')
plt.plot(newX.T[200-69]/1000,newZ.T[200-69]/newZ.T[200-69].max(),'r')
plt.plot(newX.T[200-69]/1000,newZtf.T[200-69]/newZtf.T[200-69].max(),'m')
plt.xlabel('X (km)')
plt.ylabel('Normalised Kernel (P wave)')
plt.text(100,0.9,'(d)')
plt.legend()
plt.xlim(0,25*111)
plt.savefig('Cross_kernel.pdf')



#plt.xlim(0,400)

In [ ]:

def plot_kernel_different(modeldifffile,kernellist,misfitname,figname,amplitude):
    fig, axes = plt.subplots(nrows=6, ncols=1,figsize=(15,18))
    model=np.loadtxt(modeldifffile)
    diff_3d=model.T[2].reshape(200,720)
    axes[0].imshow((diff.T)*100,animated=True, cmap=cm.seismic_r, interpolation='nearest', vmin=-2, vmax=2,\
           extent=[0,30*111,0,900],aspect='auto')
    
    for _i,filename in enumerate(kernellist):
        print(_i)
        #ax.subplot()
        data = np.loadtxt(filename)

    # checks data
        if data.ndim != 2:
        #print "Error: wrong data dimension for kernel file",data.ndim
            sys.tracebacklimit=0
            raise Exception("Invalid data dimension")

    # checks array
        if len(data[1,:]) != 5:
        #print "data shape  : ",data.shape
        #print "data lengths: ",len(data[:,1]),len(data[1,:])
        #print "Error: wrong data format for kernel file",data.shape
            sys.tracebacklimit=0
            raise Exception("Invalid data format")

        x = data[:,0]
        y = data[:,1]
      #  z1 = data[:,2] # e.g. rho
        z2 = data[:,3] # e.g. alpha
       # z3 = data[:,4] # e.g. beta

        X, Y, Z = grid(x,y,z2)
        colormap = 'RdBu'
        #print(np.max(np.abs(Z)))
        #maxx=np.log10(np.max(np.abs(Z)))
        

        im = axes[_i+1].imshow(Z*amplitude[_i+1]/np.max(np.abs(Z)),vmax=1, vmin=-1, extent=[0,25*111,0,800],cmap=colormap,aspect='auto')
               # im = ax.imshow(Z,vmax=total_max, vmin=total_min,extent=[0,45*111,0,1000],cmap=colormap.reversed())
        axes[_i+1].plot([11*111,14*111,14*111,11*111,11*111],[140,140,230,230,140],'black')
        axes[_i+1].plot(666,121,'o',color='black') #1
        axes[_i+1].plot(666,221,'o',color='black') #2
        axes[_i+1].plot(666,321,'o',color='black') #3
        axes[_i+1].plot(1400,321,'o',color='black') #4
        axes[_i+1].plot(2200,121,'o',color='black') #5 
        axes[_i+1].plot(2200,221,'o',color='black') #6 
        axes[_i+1].plot(2200,321,'o',color='black') #7
        axes[_i+1].plot(1100,321,'o',color='black') #8
        axes[_i+1].plot(1700,321,'o',color='black') #9
        axes[_i+1].plot(1100,221,'o',color='black') #10
        axes[_i+1].plot(1700,221,'o',color='black') #11
        axes[_i+1].plot(1100,121,'o',color='black') #13
        axes[_i+1].plot(1700,121,'o',color='black') #12
        receiver_x=np.linspace(555,2220,60)
        receiver_z=1e-3*np.zeros(60)
        axes[_i+1].plot(receiver_x,receiver_z, 'vk',markersize=12)
        #axes[_i].plot([0,45*111],[410,410],'black')
       # axes[_i].plot([0,45*111],[660,660],'black')
        #axes[_i].text(10,390,'410 km', fontsize=9)
        #axes[_i].text(10,640,'660 km', fontsize=9)
        axes[_i+1].set_ylim(0,400)
        axes[_i+1].set_xlim(0,25*111)
        axes[_i+1].invert_yaxis()
        axes[_i+1].set_xlabel('X (km)',fontsize=14)
        axes[_i+1].set_ylabel('Depth (km)',fontsize=14)
        axes[_i+1].text(2000,365,misfitname[_i],fontsize=14)
        axes[_i+1].text(100,300,'amplification factor:'+ str(amplitude[_i]), fontsize=14)
        #axes[_i].set_xlim(1000,3500)
    fig.colorbar(im, ax=axes.ravel().tolist(),shrink=0.5,label='Normalised Summed Misfit Kernel (P wave)')
    fig.savefig(figname)
       #axes[_i].invert_yaxis()

In [ ]:
kernellist=['smooth_summed_gradient_tf.dat','smooth_summed_gradient_ccc.dat','smooth_summed_gradient_l2.dat','smooth_summed_gradient_cc.dat','smooth_summed_gradient_mt.dat']
misfitname=['TF phase shift','CCC misfit','waveform L2','CC time shift','Multitaper time shift']
amplitude=np.array([2,1,1.2,1,1.2])
plot_kernel_different(kernellist,misfitname,'summed_kernel_compare_full.pdf',amplitude)